In [3]:
import os
from fastai.vision.all import *
import pandas as pd

# Base path to the data folder
base_data_path = Path('data')

# Initialize an empty DataFrame to store all the data
combined_df = pd.DataFrame()

# List all the folders in the base directory
folders = [f for f in os.listdir(base_data_path) if os.path.isdir(base_data_path/f)]

# Loop through each folder and load CSVs and image paths
for folder in folders:
    folder_path = base_data_path/folder
    
    # Load the CSV file (assumed to be named 'controls.csv')
    csv_file = folder_path/'controls.csv'
    if csv_file.exists():
        df = pd.read_csv(csv_file)
        
        # Get all image files that exist in the folder
        image_files = {f for f in os.listdir(folder_path) if f.endswith('.jpg')}
        
        # Dynamically match each row with an actual image filename
        def get_image_filename(i):
            # Construct the expected filename
            filename = f'frame_{i}.jpg'
            # Only return the filename if it exists in the folder
            if filename in image_files:
                return folder_path/filename
            else:
                return None

        # Apply the function to create a new column with actual image paths
        df['frame'] = df.index.map(get_image_filename)
        
        # Filter out rows where no corresponding image file was found
        df = df[df['frame'].notnull()]
        
        # Combine with the main DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# If combined_df is empty, raise an error
if combined_df.empty:
    raise FileNotFoundError("No valid image files found. Please check your data folders.")

# Define a function to get the control inputs for each image (as a tensor)
def get_y(row):
    return tensor([row['steering'], row['throttle'], row['brake']])

# Define a function to get the image path for each row
def get_x(row):
    return row['frame']  # Path is already constructed in 'combined_df'

# DataBlock for regression task
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),  # Images as input, continuous labels as output
    get_x=get_x,  # Get image path
    get_y=get_y,  # Get control inputs as a tensor
    splitter=RandomSplitter(valid_pct=0.2),  # Split train/validation
    item_tfms=Resize((395, 222)),  # Resize images
    batch_tfms=aug_transforms()  # Apply augmentations during training
)

# Create DataLoaders
dls = dblock.dataloaders(combined_df)

# Define the learner with a pretrained ResNet model
learn = vision_learner(
    dls, 
    resnet34, 
    loss_func=MSELossFlat(),
    y_range=(-1., 1.)  # Adjust as needed based on the range of your control inputs
)

learn.to_fp16()

# Fine-tune the model
print("Training the model")
learn.fine_tune(5)

# Evaluate the model on the validation set
print("Evaluating the model")
learn.validate()

# Save the entire model for future use
learn.export('trackmania_learner_fp16.pkl')


Training the model


c:\Users\robve.LAPTOP-PCBR1UJF\AppData\Local\Programs\Python\Python310\lib\site-packages\fastai\callback\fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
c:\Users\robve.LAPTOP-PCBR1UJF\AppData\Local\Programs\Python\Python310\lib\site-packages\fastai\callback\fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,time
0,0.676231,0.377373,02:04


epoch,train_loss,valid_loss,time
0,0.564543,0.311269,04:02
1,0.511093,0.270713,03:59
2,0.447131,0.208899,04:12
3,0.396775,0.187151,03:59
4,0.362334,0.186246,03:56


Evaluating the model
